<div style="background-color: lightblue; color: black; padding: 10px; font-weight: bold; font-size: 15px;\"> Current weather in Hamburg, Düsseldorf, Berlin and Munic</div>

In [24]:
# module import
import numpy as np
import pandas as pd

import requests
from dotenv import load_dotenv

import os
import json

<div style="background-color: lightblue; color: black; padding: 10px; font-weight: bold; font-size: 15px;\"> Getting geo spacial coordinates per API</div>

In [2]:
# getting latitude and longitude of relevant cities by geocoding API
# safe url
url = 'http://api.openweathermap.org/geo/1.0/direct?'
load_dotenv()

True

In [3]:
# define list of cities
cities = ['Hamburg', 'Düsseldorf', 'Berlin', 'München']

# define empty dict for geo spacial coordinates
cords = {}

# using for loop to define parameters for each city and query its coordinates
for city in cities:
    
    # parameter dictionary 
    parameters = {
    'appid': os.getenv('openweather_api_key'),
    'q': f'{city},DE'
    }

    #query data
    with requests.get(url, params=parameters) as r:
        # create json object
        json_object = json.loads(r.content)
        # get and print latitude and longitude for each city
        lat_value = json_object[0]['lat']
        print('Latidude', city, ':', lat_value)
        lon_value = json_object[0]['lon']
        print('Longitude', city, ':', lon_value)
        # add to cords dictionary
        cords[city] = [lat_value, lon_value]

    

Latidude Hamburg : 53.550341
Longitude Hamburg : 10.000654
Latidude Düsseldorf : 51.2254018
Longitude Düsseldorf : 6.7763137
Latidude Berlin : 52.5170365
Longitude Berlin : 13.3888599
Latidude München : 48.1371079
Longitude München : 11.5753822


<div style="background-color: lightblue; color: black; padding: 10px; font-weight: bold; font-size: 15px;\"> Getting weather data per API - Munic only</div

In [4]:
# set url for city weather data
url = 'https://api.openweathermap.org/data/2.5/weather?'
load_dotenv()

True

In [5]:
# setting parameters for munic only
parameters = {
    'appid': os.getenv('openweather_api_key'),
    'lat': 53.550341,
    'lon': 10.000654,
    'lang': 'en', 
    'units': 'metric'
    }

In [ ]:
# getting data for munic
r_mu = requests.get(url, parameters)
weather_mu = r_mu.json()

Now we have a nested json file.

In [7]:
# Flatten json and save in df_mu
df_mu = pd.json_normalize(weather_mu, sep="_")
df_mu

,weather,base,visibility,dt,timezone,id,name,cod,coord_lon,coord_lat,...,main_pressure,main_humidity,wind_speed,wind_deg,clouds_all,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset
0,"[{'id': 301, 'main': 'Drizzle', 'description':...",stations,4000,1697790062,7200,2911298,Hamburg,200,10.0007,53.5503,...,992,93,8.75,80,75,1,1263,DE,1697781320,1697818443


There is still a nested dict in the dataframe within the weather column, which has to be transformed to columns as well.

In [8]:
# flatten the whole json file
weather_mu = pd.json_normalize(weather_mu, 
                                  sep="_", 
                                  record_path="weather", # name the colum which is nested deeper
                                  meta=[["coord", "lon"], # structure of the remaining json
                                        ["coord", "lat"], # add a prefix to record_path
                                        "base",
                                        ["main", "temp"],
                                        ["main", "feels_like"],
                                        ["main", "temp_min"],
                                        ["main", "temp_max"], 
                                        ["main", "pressure"], 
                                        ["main", "humidity"], 
                                        "visibility", 
                                        ["wind", "speed"], 
                                        ["wind", "deg"], 
                                        ["clouds", "all"], 
                                        "dt", 
                                        ["sys", "type"], 
                                        ["sys", "id"],
                                        ["sys", "country"],
                                        ["sys", "sunrise"],
                                        ["sys", "sunset"],
                                        "timezone",
                                        "id",
                                        "name",
                                        "cod"], 
                                  record_prefix="weather_")# add a prefix to record_path

In [9]:
weather_mu

,weather_id,weather_main,weather_description,weather_icon,coord_lon,coord_lat,base,main_temp,main_feels_like,main_temp_min,...,dt,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset,timezone,id,name,cod
0,301,Drizzle,drizzle,09d,10.0007,53.5503,stations,5.91,1.13,5.16,...,1697790062,1,1263,DE,1697781320,1697818443,7200,2911298,Hamburg,200
1,701,Mist,mist,50d,10.0007,53.5503,stations,5.91,1.13,5.16,...,1697790062,1,1263,DE,1697781320,1697818443,7200,2911298,Hamburg,200


As we can see, there can be multiple values for one column, so rows are duplicated. That has to be kept in mind. First going on with handling all cities.

<div style="background-color: lightblue; color: black; padding: 10px; font-weight: bold; font-size: 15px;\"> Getting weather data per API - all cities</div

In [10]:
#getting data for all cities
data_cities = []

# defining parameter dictonary
for city, cord in cords.items():
    parameters = {
    'appid': os.getenv('openweather_api_key'),
    'lat': cords[city][0],
    'lon': cords[city][1],
    'lang': 'en', 
    'units': 'metric'
    }

    # query data
    with requests.get(url, params=parameters) as r:
        # create json object
        data_city = r.json()
        data_cities.append(data_city)


In [17]:
# empty df
df = pd.DataFrame()

# normalizing all json items in cities list and concat them into one df
for item in data_cities:
      df_city = pd.json_normalize(item, 
                                  sep="_", 
                                  record_path="weather", # name the colum which is nested deeper
                                  meta=[["coord", "lon"], # structure of the remaining json
                                        ["coord", "lat"], # add a prefix to record_path
                                        "base",
                                        ["main", "temp"],
                                        ["main", "feels_like"],
                                        ["main", "temp_min"],
                                        ["main", "temp_max"], 
                                        ["main", "pressure"], 
                                        ["main", "humidity"], 
                                        "visibility", 
                                        ["wind", "speed"], 
                                        ["wind", "deg"], 
                                        ["clouds", "all"], 
                                        "dt", 
                                        ["sys", "type"], 
                                        ["sys", "id"],
                                        ["sys", "country"],
                                        ["sys", "sunrise"],
                                        ["sys", "sunset"],
                                        "timezone",
                                        "id",
                                        "name",
                                        "cod"], 
                                  record_prefix="weather_")# add a prefix to record_path
      
      df = pd.concat([df, df_city], ignore_index=True)

In [26]:
df.head(2)

,weather_id,weather_main,weather_description,weather_icon,coord_lon,coord_lat,base,main_temp,main_feels_like,main_temp_min,...,dt,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset,timezone,id,name,cod
0,301,Drizzle,drizzle,09d,10.0007,53.5503,stations,5.91,1.13,5.16,...,1697790677,1,1263,DE,1697781320,1697818443,7200,2911298,Hamburg,200
1,701,Mist,mist,50d,10.0007,53.5503,stations,5.91,1.13,5.16,...,1697790677,1,1263,DE,1697781320,1697818443,7200,2911298,Hamburg,200


Again, we can see the multiplication of series as there are multiple values for the weather columns. Since this data is not used, we only keep every first row per city and then drop the superfluous rows.

In [19]:
# replace 'Mitte (wich is a part of Berlin) with Berlin
df['name'] = df['name'].replace('Mitte', 'Berlin')

In [ ]:
filtered_df = df[~df['name'].duplicated(keep='first')]
filtered_df = filtered_df.drop(columns=['weather_id', 'weather_main', 'weather_description', 'weather_icon'])


In [25]:
# safe dataframe as pkl
filtered_df.to_pickle('data/weather_all_cities.pkl')